# Building a Movie Recommendation System in PySpark - Lab Code-along
![images of vhs tapes on shelf](img/movies.jpg)

## Introduction

In this last lab, we will implement a a movie recommendation system using Alternating Least Squares (ALS) in the Spark programming environment.<br> Spark's machine learning libraray `ml` comes packaged with a very efficient imeplementation of the ALS algorithm. 

The lab will require you to put into practice your spark programming skills for creating and manipulating pyspark DataFrames. We will go through a step-by-step process into developing a movie recommendation system using ALS and pyspark using the MovieLens Dataset.

Note: You are advised to refer to [PySpark Documentation](http://spark.apache.org/docs/2.2.0/api/python/index.html) heavily for completing this lab as it will introduce a few new methods. 

## Objectives

You will be able to:

* Identify the key components of the ALS 
* Demonstrate an understanding on how recommendation systems are being used for personalization of online services/products
* Parse and filter datasets into Spark DataFrame, performing basic feature selection
* Run a brief hyper-parameter selection activity through a scalable grid search
* Train and evaluate the predictive performance of recommendation system
* Generate predictions from the trained model

## Building a Recommendation System

We have seen how recommender/Recommendation Systems have played an  integral parts in the success of Amazon (Books, Items), Pandora/Spotify (Music), Google (News, Search), YouTube (Videos) etc.  For Amazon these systems bring more than 30% of their total revenues. For Netflix service, 75% of movies that people watch are based on some sort of recommendation.

> The goal of Recommendation Systems is to find what is likely to be of interest to the user. This enables organizations to offer a high level of personalization and customer tailored services.

### We sort of get the concept

For online video content services like Netflix and Hulu, the need to build robust movie recommendation systems is extremely important. An example of recommendation system is such as this:

1.    User A watches Game of Thrones and Breaking Bad.
2.    User B performs a search query for Game of Thrones.
3.    The system suggests Breaking Bad to user B from data collected about user A.


This lab will guide you through a step-by-step process into developing such a movie recommendation system. We will use the MovieLens dataset to build a movie recommendation system using the collaborative filtering technique with Spark's Alternating Least Saqures implementation. After building that recommendation system, we will go through the process of adding a new user to the dataset with some new ratings and obtaining new recommendations for that user.

## Will Nightengale like Toy Story?

Collaborative filtering and matrix decomposition allows us to use the history of others ratings, along with the entire community of ratings, to answer that question.

![image1](img/collab.png)


## Person vs vegetable

It's important to realize that there are two sides to recommendation

![image2](img/item_user_based.png)

## Code for model

If we wanted, we could jump to the code right now to make this happen.

But would we understand it?
```
from pyspark.ml.evaluation import RegressionEvaluator

from pyspark.ml.recommendation import ALS, ALSModel

als = ALS(
    rank=10,
    maxIter=10,
    userCol='userId',
    itemCol='movieId',
    ratingCol='rating',
)

als_model = als.fit(movie_ratings)
```

## Documentation Station

Let's explore the [documentation](http://spark.apache.org/docs/2.4.3/api/python/pyspark.ml.html#module-pyspark.ml.recommendation) together to maybe get a better idea of what is happening. 

- which parameters make sense?
- which are completely foreign?

## Rank

What's all this rank of the factorization business?<br>
[the source code documentation](https://github.com/apache/spark/blob/master/mllib/src/main/scala/org/apache/spark/mllib/recommendation/ALS.scala) describes that variable as the "Rank of the feature matrices"

## Assumptions

Matrix decomposition is built on the theory that every individual (user, movie) score is actually the **dot product** of two separate vectors:
- user characteristics 
- movie characteristics

Wait, do you mean like gender, whether the movie is sci-fi or action? Do we have that data?

![beyonce-gif](img/beyonce.gif)

## Huh?
![what](img/what.gif)

## The hidden matricies 
![image4](img/matrix_decomp.png)

## Embeddings

Embeddings are low dimensional hidden factors for items and users.

For e.g. say we have 5 dimensional (i.e., **rank** = 5) embeddings for both items and users (5 chosen randomly, this could be any number - as we saw with PCA and dim. reduction).

For user-X & movie-A, we can say those 5 numbers might represent 5 different characteristics about the movie e.g.:

- How much movie-A is political
- How recent is the movie
- How much special effects are in movie A
- How dialogue driven is the movie
- How linear is the narrative in the movie

In a similar way, 5 numbers in the user embedding matrix might represent:

- How much does user-X like sci-fi movies
- How much does user-X like recent movies … and so on.

But we have *no actual idea* what those factors actually represent.

### If we knew the feature embeddings in advance, it would look something like this:

In [1]:
import numpy as np

# the original matrix of rankings
R = np.array([[2, np.nan, np.nan, 1, 4],
       [5, 1, 2, np.nan, 2],
       [3, np.nan, np.nan, 3, np.nan],
       [1, np.nan, 4, 2, 1]])

# users X factors
P =np.array([[-0.63274434,  1.33686735, -1.55128517],
       [-2.23813661,  0.5123861 ,  0.14087293],
       [-1.0289794 ,  1.62052691,  0.21027516],
       [-0.06422255,  1.62892864,  0.33350709]])

# factors X items
Q = np.array([[-2.09507374,  0.52351075,  0.01826269],
       [-0.45078775, -0.07334991,  0.18731052],
       [-0.34161766,  2.46215058, -0.18942263],
       [-1.0925736 ,  1.04664756,  0.69963111],
       [-0.78152923,  0.89189076, -1.47144019]])

What about that `np.nan` in the third row, last column? How will that item be reviewed by that user?

In [2]:
print(P[2])
print(Q.T[:,4])
P[2].dot(Q.T[:,4])

[-1.0289794   1.62052691  0.21027516]
[-0.78152923  0.89189076 -1.47144019]


1.9401031341455333

## Wait, I saw a transpose in there - what's the actual formula?

Terms:<br>
$R$ is the full user-item rating matrix

$P$ is a matrix that contains the users and the k factors represented as (user,factor)

$Q^T$ is a matrix that contains the items and the k factors represented as

$r̂_{u,i}$ represents our prediction for the true rating $r_{ui}$ In order to get an individual rating, you must take the dot product of a row of P and a column of Q

for the entire matrix:
$$ R = PQ^T $$ 

or for individual ratings

$$r̂_{u,i}=q_i^⊤p_u $$ 





### Let's get the whole matrix!

In [3]:
P.dot(Q.T)

array([[ 1.99717984, -0.10339773,  3.80157388,  1.00522135,  3.96947118],
       [ 4.95987359,  0.99772807,  1.9994742 ,  3.08017572,  1.99887552],
       [ 3.00799117,  0.38437256,  4.30166793,  2.96747131,  1.94010313],
       [ 0.99340337, -0.02806164,  3.96943336,  2.00841398,  1.01228247]])

### Look at those results

Are they _exactly_ correct?
![check](img/check.gif)

## ALS benefit: Loss Function

The Loss function $L$ can be calculated as:

$$ L = \sum_{u,i ∈ \kappa}(r_{u,i}− q_i^T p_u)^2 + λ( ||q_i||^2 + |p_u||^2)$$

Where $\kappa$ is the set of (u,i) pairs for which $r_{u,i}$ is known.

To avoid overfitting, the loss function also includes a regularization parameter $\lambda$. We will choose a $\lambda$ to minimize the square of the difference between all ratings in our dataset $R$ and our predictions.

There's the **least squares** part of ALS, got it!

## So now we use gradient decent, right?

![incorrect](img/incorrect.gif)

### Here comes the alternating part

ALS alternates between holding the $q_i$'s constant and the $p_u$'s constant. 

While all $q_i$'s are held constant, each $p_u$ is computed by solving the least squared problem.<br>
After that process has taken place, all the $p_u$'s are held constant while the $q_i$'s are altered to solve the least squares problem, again, each independently.<br> 
This process repeats many times until you've reached convergence (ideally).

### Changing Loss function:

First let's assume first the item vectors are fixed, we first solve for the user vectors:

$$p_u=(\sum{r{u,i}\in r_{u*}}{q_iq_i^T + \lambda I_k})^{-1}\sum_{r_{u,i}\in r_{u*}}{r_{ui}{q_{i}}}$$__
Then we hold the user vectors constant and solve for the item vectors

$$q_i=(\sum{r{u,i}\in r_{i*}}{p_up_u^T + \lambda I_k})^{-1}\sum_{r_{u,i}\in r_{u*}}{r_{ui}{p_{u}}}$$__
This process repeats until convergence

# Review
What levers do we have available to adjust?
![lever-choice](img/levers.jpeg)

- Pros and cons of large rank?
- Pros and cons of lambda size?
- Iterations?

# Enough - let's get to the data

### Importing the Data
To begin with:
* initialize a SparkSession object
* import the dataset found at './data/ratings.csv' into a pyspark DataFrame

In [4]:
# import necessary libraries
!env | grep PYTHONPATH

In [5]:
import pyspark

spark = (pyspark.sql.SparkSession.builder 
  .master("local[*]")
  .getOrCreate())

In [6]:
!ls data/

movies.csv  ratings.csv


In [7]:
!file data/ratings.csv

data/ratings.csv: ASCII text, with CRLF line terminators


In [8]:
!head data/ratings.csv

In [9]:
# read in the dataset into pyspark DataFrame
movie_ratings = spark.read.csv('data/ratings.csv',
                               inferSchema=True,
                               header=True)

Check the data types of each of the values to ensure that they are a type that makes sense given the column.

In [10]:
movie_ratings.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



But if they were ever incorrectly assigned, here's how we fix it:

In [11]:
from pyspark.sql.types import (
    ArrayType,
    AtomicType,
    BinaryType,
    BooleanType,
    ByteType,
    CloudPickleSerializer,
    DataType,
    DataTypeSingleton,
    DateConverter,
    DateType,
    DatetimeConverter,
    DecimalType,
    DoubleType,
    FloatType,
    FractionalType,
    IntegerType,
    IntegralType,
    JavaClass,
    LongType,
    MapType,
    NullType,
    NumericType,
    Row,
    ShortType,
    SparkContext,
    StringType,
    StructField,
    StructType,
    TimestampType,
    UserDefinedType,
)

In [12]:
schema = StructType(
    [
        StructField('userId', IntegerType()),
        StructField('movieId', IntegerType()),
        StructField('rating', FloatType()),
        StructField('timestamp', LongType()),
    ]
)

In [13]:
# read in the dataset into pyspark DataFrame
movie_ratings = spark.read.csv('data/ratings.csv',
                               inferSchema=False,
                               schema=schema,
                               header=True)

In [14]:
movie_ratings.persist()

DataFrame[userId: int, movieId: int, rating: float, timestamp: bigint]

In [15]:
movie_ratings.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: float (nullable = true)
 |-- timestamp: long (nullable = true)



In [16]:
type(movie_ratings)

pyspark.sql.dataframe.DataFrame

In [17]:
movie_ratings.show()

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
|     1|     70|   3.0|964982400|
|     1|    101|   5.0|964980868|
|     1|    110|   4.0|964982176|
|     1|    151|   5.0|964984041|
|     1|    157|   5.0|964984100|
|     1|    163|   5.0|964983650|
|     1|    216|   5.0|964981208|
|     1|    223|   3.0|964980985|
|     1|    231|   5.0|964981179|
|     1|    235|   4.0|964980908|
|     1|    260|   5.0|964981680|
|     1|    296|   3.0|964982967|
|     1|    316|   3.0|964982310|
|     1|    333|   5.0|964981179|
|     1|    349|   4.0|964982563|
+------+-------+------+---------+
only showing top 20 rows



We aren't going to need the time stamp, so we can go ahead and remove that column.

In [ ]:
movie_ratings = #what do we put here?

In [23]:
#Answer to above
movie_ratings = movie_ratings.select(movie_ratings.columns[:-1])
movie_ratings.show(5)

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|      1|   4.0|
|     1|      3|   4.0|
|     1|      6|   4.0|
|     1|     47|   5.0|
|     1|     50|   5.0|
+------+-------+------+
only showing top 5 rows



### Fitting the Alternating Least Squares Model

Because this dataset is already preprocessed for us, we can go ahead and fit the Alternating Least Squares model.

* Use the randomSplit method on the pyspark DataFrame to separate the dataset into a training and test set
* Import the ALS module from pyspark.ml.recommendation.
* Fit the Alternating Least Squares Model to the training dataset. Make sure to set the userCol, itemCol, and ratingCol to the appropriate names given this dataset. Then fit the data to the training set and assign it to a variable model. 

### Train-Test Split

Split the data into training and testing sets.

In [25]:
movie_ratings.count()

100836

In [28]:
train_data , test_data = movie_ratings.randomSplit([.8, .2])
print("Training Size: {}, Testing Size: {}".format(train_data.count(), test_data.count()))
length = movie_ratings.count()
print("Training Size: {}, Testing Size: {}".format(train_data.count()/length, test_data.count()/length))

Training Size: 80760, Testing Size: 20076
Training Size: 0.8009044388908723, Testing Size: 0.19909556110912768


In [29]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS, ALSModel

als = ALS(
    rank=10,
    maxIter=10,
    userCol='userId',
    itemCol='movieId',
    ratingCol='rating',
)

In [30]:
# Build the recommendation model using ALS on the training data
# fit the ALS model to the training set

als_model = als.fit(train_data)

Now you've fit the model, and it's time to evaluate it to determine just how well it performed.

* Import `RegressionEvalutor` from pyspark.ml.evaluation [documentation](http://spark.apache.org/docs/2.4.3/api/python/pyspark.ml.html#pyspark.ml.evaluation.RegressionEvaluator)
* Generate predictions with your model for the train and test set by using the `transform` method on your ALS model
* Evaluate your model and print out the RMSE from your test set [options for evaluating regressors](http://spark.apache.org/docs/2.4.3/api/python/pyspark.ml.html#pyspark.ml.evaluation.RegressionEvaluator.metricName)

In [31]:
ALSModel.transform?

In [61]:
train_predictions = als_model.transform(train_data)
test_predictions = als_model.transform(test_data)

In [62]:
train_predictions.persist()
test_predictions.persist()

DataFrame[userId: int, movieId: int, rating: float, prediction: float]

In [63]:
movie_ratings.show(1)

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|      1|   4.0|
+------+-------+------+
only showing top 1 row



In [65]:
train_predictions.show(1)

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|   191|    148|   5.0| 4.9202523|
+------+-------+------+----------+
only showing top 1 row



In [64]:
test_predictions.show(1)

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|   133|    471|   4.0| 2.9283142|
+------+-------+------+----------+
only showing top 1 row



In [37]:
user_factors = als_model.userFactors

In [38]:
user_factors

DataFrame[id: int, features: array<float>]

In [39]:
item_factors = als_model.itemFactors

In [66]:
evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="rating", metricName="rmse")
evaluator.evaluate(train_predictions)

0.5654571135030395

In [69]:
evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="rating", metricName="rmse")
evaluator.evaluate(test_predictions.filter(test_predictions.prediction.isNotNull()))

nan

> **Warning!**
Spark Considers Null Values and NaN (Not a Number) different entities. See the code below, where the NaN values from our prediction are not filtered out. See this [stack overflow thread](https://stackoverflow.com/questions/43882699/differences-between-null-and-nan-in-spark-how-to-deal-with-it#43883025) for further details.

In [80]:
test_predictions.prediction.is

Column<b'(prediction IS NOT NULL)'>

In [84]:
test_predictions.filter(test_predictions.prediction.isNotNull()).show(2500)

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|   133|    471|   4.0| 2.9283142|
|   372|    471|   3.0| 3.4082422|
|   500|    471|   1.0| 1.4433138|
|   462|    471|   2.5| 2.0555644|
|   387|    471|   3.0| 2.7757351|
|   217|    471|   2.0|  2.620111|
|   555|    471|   3.0| 4.2340193|
|   171|    471|   3.0| 4.7378817|
|   312|    471|   4.0| 3.5492857|
|   411|    471|   4.0| 3.1242478|
|   287|    471|   4.5|  2.982832|
|    32|    471|   3.0| 4.3000693|
|   260|    471|   4.5| 3.4367647|
|   608|    833|   0.5|  2.005748|
|   111|   1088|   3.0|  3.139306|
|    20|   1088|   4.5|  3.319328|
|    64|   1088|   4.0| 2.9902453|
|   554|   1088|   5.0|   3.97138|
|   286|   1088|   3.5| 3.5916922|
|   594|   1088|   4.5|  4.321806|
|   221|   1088|   3.0| 2.7761447|
|   414|   1088|   3.0|  3.023644|
|    68|   1088|   3.5| 3.4387605|
|   517|   1088|   1.0| 3.1206899|
|   325|   1238|   4.0|  4.507472|
|   599|   1342|   2

In [86]:
from pyspark.sql.functions import isnan
evaluator.evaluate(test_predictions.where(isnan("prediction")))

nan

In [90]:
print(test_predictions.count())
test_predictions = test_predictions.na.drop()
print(test_predictions.count())
evaluator.evaluate(test_predictions)

20076
19285


0.8908312639058603

### Important Question

Will Billy like movie m?

In [49]:
billy_row = user_factors[user_factors['id'] == 10].first()
billy_factors = np.array(billy_row['features'])

In [50]:
m_row = item_factors[item_factors['id'] == 296].first()
m_factors = np.array(m_row['features'])

In [51]:
billy_factors

array([ 0.63686609,  0.45804632,  0.28430074,  1.02743554, -0.80457139,
        0.28032553,  0.20997393,  1.38921404,  0.33457235, -0.17821293])

In [52]:
m_factors

array([ 0.58915579,  1.13703477, -1.38429129,  1.181077  ,  0.18176536,
        1.1247915 ,  0.45762929, -0.02493427,  0.33827844,  0.20756954])

In [53]:
billy_factors @ m_factors

2.022656176543938

In [54]:
billy_preds = predictions[predictions['userId'] == 10]

In [55]:
billy_preds.sort('movieId').show()

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|    10|    296|   1.0|  2.022656|
|    10|    356|   3.5| 3.7200947|
|    10|    588|   4.0| 3.3394587|
|    10|    912|   4.0| 3.4187894|
|    10|   2671|   3.5| 3.3281434|
|    10|   4306|   4.5|  3.319521|
|    10|   5957|   3.0| 3.2033124|
|    10|   6535|   4.0| 2.3917892|
|    10|   7293|   3.5|   4.02743|
|    10|  31433|   2.5| 3.8346786|
|    10|  40819|   4.0|  3.014394|
|    10|  49286|   5.0| 3.8277862|
|    10|  51662|   3.0|  3.049507|
|    10|  51705|   4.5| 3.0408254|
|    10|  56367|   3.5| 3.4707341|
|    10|  56949|   3.0| 3.4603763|
|    10|  64969|   0.5| 3.5690844|
|    10|  68954|   4.0| 3.4042344|
|    10|  70293|   0.5|  2.857835|
|    10|  72011|   2.5| 2.6984928|
+------+-------+------+----------+
only showing top 20 rows



In [57]:
!grep 296 < data/movies.csv

## Okay, what *will* Billy like?

In [58]:
recs = als_model.recommendForAllUsers(numItems=10)

In [59]:
recs[recs['userId']==10].first()['recommendations']

[Row(movieId=7018, rating=5.385290145874023),
 Row(movieId=32892, rating=5.0265421867370605),
 Row(movieId=6732, rating=4.86475133895874),
 Row(movieId=67618, rating=4.6341376304626465),
 Row(movieId=8869, rating=4.600409984588623),
 Row(movieId=74946, rating=4.589346885681152),
 Row(movieId=71579, rating=4.523812294006348),
 Row(movieId=3086, rating=4.487799644470215),
 Row(movieId=6944, rating=4.477414131164551),
 Row(movieId=140110, rating=4.477058410644531)]

In [60]:
!grep 3086 < data/movies.csv

## Objective Review

* Identify the key components of the ALS 
* Demonstrate an understanding on how recommendation systems are being used for personalization of online services/products
* Parse and filter datasets into Spark DataFrame, performing basic feature selection
* Run a brief hyper-parameter selection activity through a scalable grid search
* Train and evaluate the predictive performance of recommendation system
* Generate predictions from the trained model

## Some great technical resources:

- [Stanford ALS Theory and Implementation](http://stanford.edu/~rezab/classes/cme323/S15/notes/lec14.pdf)
- [The Netflix Recommendation Project](https://www.netflixprize.com/assets/GrandPrize2009_BPC_BellKor.pdf)